# Trabalho prático AC

## Trabalho realizado por:

Margarida Vila-chã, 20210504

Miguel Duarte Silva, 20210504

Sebastião Lessa, 202103238


## Algoritmo escolhido e breve explicação:

O algoritmo escolhido pelo nosso grupo foi o Random Forrest.

Este algoritmo é um algoritmo de aprendizagem computacional supervisionado. Ele usa uma combinação de árvores de decisão aleatórias. O processo começa com a criação de um grande número de árvores (floresta), em que cada uma das árvores criadas é treinada com uma amostra diferente. Isto permite a criação de árvores suficientemente diferentes, o que leva a que elas juntas consigam lidar com uma ampla variedade de situações, reduzindo o risco de overffitting.

Baseamos-nos nos seguintes data-sets:

- [credit-g](https://www.openml.org/search?type=run&id=591&collections.id=99&run_task.task_id=31&sort=date)
- [breast-w](https://www.openml.org/search?type=run&id=7413&collections.id=99&run_task.task_id=15)
- [qsar-biodeg](https://www.openml.org/search?type=run&id=519587&collections.id=99&run_task.task_id=9957)

## Alteração escolhida
A alteração que o nosso grupo decidiu fazer no codigo fonte obtido foi a de considerar a utilização de outros tipos de critérios de divisão. O critério pré definido no nosso código fonte é o de entropia, por isso vamos testar com o gini e com o classification error.

## Modelo implementado, sem alteração

In [223]:
from collections import Counter

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feature, best_thresh, left, right)

    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # calculate the information gain
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold

    def _information_gain(self, y, X_column, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)

        # create children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # calculate the weighted avg. entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r

        # calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p > 0])

    def _most_common_label(self, y):
        #print(y)
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

In [65]:
class RF_local:
    def __init__(self, n_trees=10, max_depth=10, min_samples_split=2, n_feature=None, class_weights=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_features = n_feature
        self.class_weights = class_weights
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        if self.class_weights is None:
            self.class_weights = np.ones(len(np.unique(y)))
        for _ in range(self.n_trees):
            tree = DecisionTree(max_depth=self.max_depth,
                                min_samples_split=self.min_samples_split,
                                n_features=self.n_features)
            X_sample, y_sample = self._bootstrap_samples(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def _bootstrap_samples(self, X, y):
        n_samples = X.shape[0]
        weights = self.class_weights[y] / np.sum(self.class_weights[y])
        idxs = np.random.choice(n_samples, n_samples, replace=True, p=weights)
        return X[idxs], y[idxs]

    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

    def _update_class_weights(self, y):
        n_classes = len(self.class_weights)
        class_counts = np.bincount(y, minlength=n_classes)
        if np.any(class_counts == 0):
            warnings.warn("Some classes have no samples in the training data.")
        self.class_weights = np.sqrt(np.sum(class_counts)) / np.sqrt(class_counts)
        self.class_weights /= np.sum(self.class_weights)

    def predict(self, X):
        predictions = np.zeros((X.shape[0]), len(self.class_weights))
        for tree in self.trees:
            tree_predictions = tree.predict(X)
            predictions[np.arange(X.shape[0]),tree_predictions.argmax(axis=1)] += self.class_weights
        final_predictions = np.argmax(predictions,axis=1)
        return final_predictions

## Import das bibliotecas necessárias:

In [66]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from random import random

from sklearn import datasets
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestClassifier as RF


## Testes para cada dataset

### DATASET 1 - Cancro da mama

In [226]:
dataset = pd.read_csv('breast.csv')

In [227]:
dataset.head()

,id,Clump_Thickness,Cell_Size_Uniformity,Cell_Shape_Uniformity,Marginal_Adhesion,Single_Epi_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1,5,1,1,1,2,1,3,1,1,benign
1,2,5,4,4,5,7,10,3,2,1,benign
2,3,3,1,1,1,2,2,3,1,1,benign
3,4,6,8,8,1,3,4,3,7,1,benign
4,5,4,1,1,3,2,1,3,1,1,benign


In [228]:
for column in ['Class']:
    le = LabelEncoder()
    dataset[column] = le.fit_transform(dataset[column])

In [229]:
dataset.head()

,id,Clump_Thickness,Cell_Size_Uniformity,Cell_Shape_Uniformity,Marginal_Adhesion,Single_Epi_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1,5,1,1,1,2,1,3,1,1,0
1,2,5,4,4,5,7,10,3,2,1,0
2,3,3,1,1,1,2,2,3,1,1,0
3,4,6,8,8,1,3,4,3,7,1,0
4,5,4,1,1,3,2,1,3,1,1,0


In [230]:
from sklearn.datasets import load_digits
digits=load_digits()

In [231]:
dir(digits)

['DESCR', 'data', 'feature_names', 'frame', 'images', 'target', 'target_names']

In [232]:
df=pd.DataFrame(digits.data)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,9.0,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0


In [233]:
df['target']=digits.target
df.head()

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,target
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0,1
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0,2
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0,3
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0,4


In [75]:
def test_model(df, model):
    X=df.iloc[:,1:-1].values
    y=df.iloc[:,-1].values

    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=42)
    
    model.fit(X_train, y_train)
    
    # print(df.info)
    
    # print(forest.predict(X_test))
    
    print("Accuracy:", accuracy_score(y_test, model.predict(X_test)))


#### Teste 1 - Random Forest (sklearn) com o critério default (gini)

In [76]:
test_model(df, RF())

Accuracy: 0.9777777777777777


#### Teste 2 - Random Forest (sklearn) com o critério entropy

In [77]:
test_model(df, RF(criterion='entropy'))

Accuracy: 0.975925925925926


#### Teste 3 - Random Forest (local) com o critério default (gini)

In [78]:
test_model(df, RF_local())

<class 'numpy.ndarray'>


TypeError: Cannot interpret '10' as a data type

#### Teste 4 - Random Forest (local) com o critério entropy

In [ ]:
test_model(df, RF_local(criteria='entropy'))

## Teste com o modelo com alterações, usando o gini - Dataset 1

## Teste com o modelo com alterações, usando o classifcation error - Dataset 1

# Dataset 2

## Teste com o sk learn com entropia - Dataset 2

In [ ]:
df = pd.read_csv('biodeg.csv')

In [ ]:
df.head()

In [ ]:
for column in ['Class']:
    le = LabelEncoder()
    dataset[column] = le.fit_transform(df[column])

In [ ]:
df.head()

In [ ]:
from sklearn.datasets import load_digits
digits=load_digits()

In [ ]:
X=df.iloc[:,1:-1].values
y=df.iloc[:,-1].values

X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.2, random_state=0
)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(criterion='entropy')
model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

def calculate_accuracy(model, X, y):
    # Make predictions using the model
    predictions = model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, predictions)

    return accuracy

accuracy = calculate_accuracy(model, X, y)
print("Accuracy:", accuracy)

## Teste com o algortimo com entropy, sem alterações - Dataset 2

In [ ]:
from DecisionTree import DecisionTree
import numpy as np
import pandas as pd





# Extract the features (X) and target (y) from the DataFrame
X=df.iloc[:,1:-1].values
y=df.iloc[:,-1].values

X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.2, random_state=0
)

# Instantiate the RandomForest class
forest = RandomForest()

# Fit the random forest to the data
forest.fit(X_train, y_train)

# Make predictions
predictions = forest.predict(X_test)

# Print the predictions
print(predictions)

In [ ]:
from sklearn.metrics import accuracy_score

def calculate_accuracy(model, X, y):
    # Make predictions using the model
    predictions = model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, predictions)

    return accuracy

accuracy = calculate_accuracy(forest, X, y)
print("Accuracy:", accuracy)

## Teste com o metodo gini - Dataset 2

## Teste com o metodo classification error - Dataset 2

# Teste com o metodo sk learn - Dataset 3

In [ ]:
df=pd.read_csv('spambase.csv')

In [ ]:
df.head()

In [ ]:
X=df.iloc[:,1:-1].values
y=df.iloc[:,-1].values

X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.2, random_state=0
)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(criterion='entropy')
model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

def calculate_accuracy(model, X, y):
    # Make predictions using the model
    predictions = model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, predictions)

    return accuracy

accuracy = calculate_accuracy(model, X, y)
print("Accuracy:", accuracy)

## Teste com o algoritmo base - Dataset 3

In [ ]:
from DecisionTree import DecisionTree
import numpy as np
import pandas as pd


# Extract the features (X) and target (y) from the DataFrame
X=df.iloc[:,1:-1].values
y=df.iloc[:,-1].values

X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.2, random_state=0
)

# Instantiate the RandomForest class
forest = RandomForest()

# Fit the random forest to the data
forest.fit(X_train, y_train)

# Make predictions
predictions = forest.predict(X_test)

# Print the predictions
print(predictions)
